# Large-Scale Elastic Training Practice

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mmcmanus1/elastic-training-harness/blob/main/notebooks/elastic_training_large.ipynb)

A comprehensive practice notebook with:
- **~100M parameter model** (3.5x larger than basic demo)
- **Real data** - WikiText-2 dataset with GPT-2 tokenization
- **500 training steps** with validation tracking
- **Full feature demonstration** - checkpointing, scaling, recovery

Designed for Google Colab T4 GPU (16GB VRAM).

## 1. Setup

Clone the repository and install dependencies.

In [ ]:
# Setup: Clone and install elastic-training-harness
# Clean up any existing installation (safe to re-run)
!rm -rf /content/elastic-training-harness

# Clone the repository
!git clone https://github.com/mmcmanus1/elastic-training-harness.git

# Install the package and additional dependencies (non-editable for Colab compatibility)
%cd /content/elastic-training-harness
!pip install . --quiet
!pip install datasets transformers --quiet
%cd /content

# Force Python to recognize newly installed package
import importlib
import sys
if 'elastic_harness' in sys.modules:
    del sys.modules['elastic_harness']
importlib.invalidate_caches()

# Verify installation
try:
    import elastic_harness
    print(f"elastic_harness {elastic_harness.__version__} installed successfully!")
except ImportError as e:
    print(f"Installation failed: {e}")
    print("Try: Runtime -> Restart runtime, then run this cell again")

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import time
import random
import os

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"GPU: {gpu_name}")
    print(f"VRAM: {gpu_memory:.1f} GB")

## 2. Import elastic_harness Components

In [ ]:
# Checkpoint management
from elastic_harness.checkpoint import (
    CheckpointConfig,
    CheckpointState,
    CheckpointManager,
    CheckpointTier,
    MemorySnapshotBackend,
    NVMeBackend,
)

# LR scaling and gradient accumulation
from elastic_harness.scaling import (
    ScalingRule,
    ScalingConfig,
    LRScalingManager,
    GradAccumulationConfig,
    GradientAccumulationManager,
    ElasticScalingManager,
)

print("elastic_harness components imported successfully!")

## 3. Load WikiText-2 Dataset

Load real text data and tokenize with GPT-2 tokenizer.

In [ ]:
from datasets import load_dataset
from transformers import GPT2Tokenizer

# Load WikiText-2
print("Loading WikiText-2 dataset...")
dataset = load_dataset("wikitext", "wikitext-2-raw-v1")

# Load GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

print(f"Train samples: {len(dataset['train'])}")
print(f"Validation samples: {len(dataset['validation'])}")
print(f"Vocab size: {tokenizer.vocab_size}")

In [ ]:
SEQ_LENGTH = 256

class WikiTextDataset(Dataset):
    """Dataset that tokenizes and chunks text into fixed-length sequences."""
    
    def __init__(self, texts, tokenizer, seq_length):
        self.seq_length = seq_length
        
        # Concatenate all text and tokenize
        all_text = " ".join([t for t in texts if t.strip()])
        tokens = tokenizer.encode(all_text, add_special_tokens=False)
        
        # Chunk into sequences
        self.chunks = []
        for i in range(0, len(tokens) - seq_length, seq_length):
            self.chunks.append(tokens[i:i + seq_length])
        
        print(f"Created {len(self.chunks)} sequences of length {seq_length}")
    
    def __len__(self):
        return len(self.chunks)
    
    def __getitem__(self, idx):
        chunk = torch.tensor(self.chunks[idx], dtype=torch.long)
        # Input is all but last token, labels shifted by 1
        return chunk[:-1], chunk[1:]

# Create datasets
print("Tokenizing and chunking train data...")
train_dataset = WikiTextDataset(
    dataset['train']['text'], 
    tokenizer, 
    SEQ_LENGTH + 1  # +1 for labels shift
)

print("Tokenizing and chunking validation data...")
val_dataset = WikiTextDataset(
    dataset['validation']['text'],
    tokenizer,
    SEQ_LENGTH + 1
)

In [ ]:
# Create DataLoaders
BATCH_SIZE = 2  # Small batch, will use gradient accumulation

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
)

print(f"Train batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

## 4. Define ~100M Parameter Model

Larger transformer model sized for T4 GPU.

In [ ]:
class LargeTransformerLM(nn.Module):
    """~100M parameter transformer language model."""

    def __init__(
        self,
        vocab_size: int = 50257,
        d_model: int = 512,
        nhead: int = 8,
        num_layers: int = 6,
        dim_feedforward: int = 2048,
        max_seq_length: int = 256,
        dropout: float = 0.1,
    ):
        super().__init__()
        self.d_model = d_model
        self.max_seq_length = max_seq_length

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_embedding = nn.Embedding(max_seq_length, d_model)
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
            batch_first=True,
            norm_first=True,  # Pre-norm for better training
        )
        self.transformer = nn.TransformerEncoder(
            encoder_layer, 
            num_layers=num_layers,
            enable_nested_tensor=False,
        )
        
        self.ln_f = nn.LayerNorm(d_model)
        self.output_proj = nn.Linear(d_model, vocab_size, bias=False)
        self.output_proj.weight = self.embedding.weight  # Tie weights

        # Initialize weights
        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, input_ids: torch.Tensor, labels=None):
        batch_size, seq_len = input_ids.shape
        
        # Create position indices
        positions = torch.arange(seq_len, device=input_ids.device).unsqueeze(0)
        positions = positions.expand(batch_size, -1)

        # Embeddings
        x = self.embedding(input_ids) + self.pos_embedding(positions)
        x = self.dropout(x)

        # Causal mask
        mask = nn.Transformer.generate_square_subsequent_mask(
            seq_len, device=input_ids.device
        )
        
        # Transformer
        x = self.transformer(x, mask=mask, is_causal=True)
        x = self.ln_f(x)

        # Output projection
        logits = self.output_proj(x)

        output = {"logits": logits}

        if labels is not None:
            loss_fct = nn.CrossEntropyLoss()
            loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
            output["loss"] = loss

        return output


# Create model and count parameters
model = LargeTransformerLM(
    vocab_size=tokenizer.vocab_size,
    d_model=512,
    nhead=8,
    num_layers=6,
    dim_feedforward=2048,
    max_seq_length=SEQ_LENGTH,
)

num_params = sum(p.numel() for p in model.parameters())
num_trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Model parameters: {num_params:,}")
print(f"Trainable parameters: {num_trainable:,}")
print(f"Model size: ~{num_params * 4 / 1e6:.1f} MB (fp32)")

## 5. Training with Full elastic_harness Features

Training loop with:
- Gradient accumulation (effective batch size 8)
- Multi-tier checkpointing (Memory + NVMe)
- Validation tracking
- Memory monitoring

In [ ]:
def get_gpu_memory_mb():
    """Get current GPU memory usage in MB."""
    if torch.cuda.is_available():
        return torch.cuda.memory_allocated() / 1e6
    return 0


@torch.no_grad()
def evaluate(model, val_loader, device, max_batches=50):
    """Run validation and return average loss."""
    model.eval()
    total_loss = 0
    num_batches = 0
    
    for i, (input_ids, labels) in enumerate(val_loader):
        if i >= max_batches:
            break
        input_ids = input_ids.to(device)
        labels = labels.to(device)
        
        outputs = model(input_ids, labels=labels)
        total_loss += outputs['loss'].item()
        num_batches += 1
    
    model.train()
    return total_loss / num_batches if num_batches > 0 else 0


def train_large_model(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    max_steps: int = 500,
    accumulation_steps: int = 4,  # Effective batch = 2 * 4 = 8
    checkpoint_interval: int = 100,
    memory_snapshot_interval: int = 25,
    val_interval: int = 100,
    log_interval: int = 25,
    chaos_enabled: bool = False,
    chaos_probability: float = 0.01,
    chaos_after_step: int = 200,
):
    """Training loop with full elastic_harness features."""

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Setup optimizer
    optimizer = torch.optim.AdamW(
        model.parameters(), 
        lr=3e-4,  # Higher LR for larger model
        weight_decay=0.01,
        betas=(0.9, 0.95),
    )

    # Setup elastic scaling manager (unified LR + accumulation)
    lr_config = ScalingConfig(
        base_lr=3e-4,
        base_batch_size=BATCH_SIZE * accumulation_steps,
        base_world_size=1,
        scaling_rule=ScalingRule.SQRT,  # Sqrt scaling for large batches
        warmup_steps=50,
    )
    lr_manager = LRScalingManager(lr_config, optimizer)

    # Setup multi-tier checkpoint backends
    memory_backend = MemorySnapshotBackend(max_snapshots=3)
    nvme_backend = NVMeBackend(base_path='/tmp/elastic_large_checkpoints')

    # Try to resume from checkpoint
    start_step = 0
    latest_path = nvme_backend.get_latest_checkpoint()
    if latest_path:
        checkpoint = nvme_backend.load(latest_path)
        model.load_state_dict(checkpoint.model_state_dict)
        optimizer.load_state_dict(checkpoint.optimizer_state_dict)
        start_step = checkpoint.step + 1
        print(f"Resumed from checkpoint at step {checkpoint.step}")
    else:
        print("Starting fresh training run")

    # Training state
    train_losses = []
    val_losses = []
    val_steps = []
    memory_usage = []
    
    model.train()
    train_iter = iter(train_loader)
    accumulated_loss = 0
    
    print(f"\nStarting training from step {start_step} to {max_steps}")
    print(f"Effective batch size: {BATCH_SIZE * accumulation_steps}")
    print(f"Chaos mode: {'ENABLED' if chaos_enabled else 'disabled'}")
    print("-" * 60)

    for step in range(start_step, max_steps):
        step_start = time.time()

        # Chaos: maybe crash (for testing recovery)
        if chaos_enabled and step > chaos_after_step:
            if random.random() < chaos_probability:
                print(f"\nCHAOS: Simulated crash at step {step}!")
                print("Run this cell again to resume from checkpoint.")
                return {
                    'train_losses': train_losses,
                    'val_losses': val_losses,
                    'val_steps': val_steps,
                    'memory_usage': memory_usage,
                    'final_step': step,
                    'crashed': True,
                }

        # Memory snapshot
        if step > 0 and step % memory_snapshot_interval == 0:
            state = CheckpointState(
                step=step,
                model_state_dict=model.state_dict(),
                optimizer_state_dict=optimizer.state_dict(),
            )
            memory_backend.save(state)

        # Gradient accumulation loop
        optimizer.zero_grad()
        for micro_step in range(accumulation_steps):
            try:
                input_ids, labels = next(train_iter)
            except StopIteration:
                train_iter = iter(train_loader)
                input_ids, labels = next(train_iter)
            
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            
            outputs = model(input_ids, labels=labels)
            loss = outputs['loss'] / accumulation_steps
            loss.backward()
            accumulated_loss += loss.item()

        # Gradient clipping and optimizer step
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        # LR warmup step
        current_lr = lr_manager.step()

        train_losses.append(accumulated_loss)
        memory_usage.append(get_gpu_memory_mb())
        accumulated_loss = 0

        # Logging
        if step % log_interval == 0:
            avg_loss = sum(train_losses[-log_interval:]) / min(log_interval, len(train_losses))
            mem_mb = memory_usage[-1]
            step_time = time.time() - step_start
            tokens_per_sec = BATCH_SIZE * accumulation_steps * SEQ_LENGTH / step_time
            print(
                f"Step {step:4d} | Loss: {avg_loss:.4f} | "
                f"LR: {current_lr:.2e} | Mem: {mem_mb:.0f}MB | "
                f"Tok/s: {tokens_per_sec:.0f}"
            )

        # Validation
        if step > 0 and step % val_interval == 0:
            val_loss = evaluate(model, val_loader, device)
            val_losses.append(val_loss)
            val_steps.append(step)
            print(f"  [Validation loss: {val_loss:.4f}]")

        # NVMe checkpoint
        if step > 0 and step % checkpoint_interval == 0:
            state = CheckpointState(
                step=step,
                model_state_dict=model.state_dict(),
                optimizer_state_dict=optimizer.state_dict(),
                metrics={'train_loss': train_losses[-1], 'val_loss': val_losses[-1] if val_losses else None},
            )
            path = f"checkpoint_step_{step:08d}.pt"
            nvme_backend.save(state, path)
            nvme_backend.cleanup_old_checkpoints(keep_last=2)
            print(f"  [Checkpoint saved: {path}]")
            memory_backend.clear()

    # Final validation
    final_val_loss = evaluate(model, val_loader, device)
    val_losses.append(final_val_loss)
    val_steps.append(max_steps)

    print("-" * 60)
    print(f"Training complete!")
    print(f"Final train loss: {train_losses[-1]:.4f}")
    print(f"Final val loss: {final_val_loss:.4f}")

    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_steps': val_steps,
        'memory_usage': memory_usage,
        'final_step': max_steps,
        'crashed': False,
    }

## 6. Run Training

Train for 500 steps. Set `chaos_enabled=True` to test crash recovery.

In [ ]:
# Create fresh model
model = LargeTransformerLM(
    vocab_size=tokenizer.vocab_size,
    d_model=512,
    nhead=8,
    num_layers=6,
    dim_feedforward=2048,
    max_seq_length=SEQ_LENGTH,
)

# Run training
results = train_large_model(
    model,
    train_loader,
    val_loader,
    max_steps=500,
    accumulation_steps=4,  # Effective batch = 8
    checkpoint_interval=100,
    memory_snapshot_interval=25,
    val_interval=100,
    log_interval=25,
    chaos_enabled=False,  # Set True to test recovery
    chaos_probability=0.02,
    chaos_after_step=200,
)

## 7. Visualize Training

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Training loss
axes[0].plot(results['train_losses'], alpha=0.7)
axes[0].set_xlabel('Step')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].grid(True, alpha=0.3)

# Validation loss
axes[1].plot(results['val_steps'], results['val_losses'], 'o-', color='orange')
axes[1].set_xlabel('Step')
axes[1].set_ylabel('Loss')
axes[1].set_title('Validation Loss')
axes[1].grid(True, alpha=0.3)

# Memory usage
axes[2].plot(results['memory_usage'], color='green', alpha=0.7)
axes[2].set_xlabel('Step')
axes[2].set_ylabel('Memory (MB)')
axes[2].set_title('GPU Memory Usage')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Initial loss: {results['train_losses'][0]:.4f}")
print(f"Final loss: {results['train_losses'][-1]:.4f}")
print(f"Loss reduction: {results['train_losses'][0] - results['train_losses'][-1]:.4f}")

## 8. Checkpoint Recovery Demo

Demonstrate loading from a saved checkpoint and continuing training.

In [ ]:
print("Checkpoint Recovery Demonstration")
print("=" * 50)

# Create a fresh model
fresh_model = LargeTransformerLM(
    vocab_size=tokenizer.vocab_size,
    d_model=512,
    nhead=8,
    num_layers=6,
    dim_feedforward=2048,
    max_seq_length=SEQ_LENGTH,
)

# Load checkpoint
nvme_backend = NVMeBackend(base_path='/tmp/elastic_large_checkpoints')
latest_path = nvme_backend.get_latest_checkpoint()

if latest_path:
    checkpoint = nvme_backend.load(latest_path)
    fresh_model.load_state_dict(checkpoint.model_state_dict)
    
    print(f"Loaded checkpoint from step {checkpoint.step}")
    print(f"Checkpoint metrics: {checkpoint.metrics}")
    
    # Verify model works
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    fresh_model = fresh_model.to(device)
    
    val_loss = evaluate(fresh_model, val_loader, device)
    print(f"Validation loss after loading: {val_loss:.4f}")
    print("\nCheckpoint recovery successful!")
else:
    print("No checkpoint found - run training first")

## 9. Topology Change Simulation

Demonstrate how elastic_harness handles dynamic world size changes.

In [ ]:
print("Simulating Topology Changes")
print("=" * 60)

# Setup with elastic scaling managers
dummy_model = LargeTransformerLM(vocab_size=1000, d_model=64, num_layers=1)  # Tiny for demo
dummy_optimizer = torch.optim.AdamW(dummy_model.parameters(), lr=3e-4)

# ElasticScalingManager combines LR and accumulation
elastic_manager = ElasticScalingManager(
    lr_config=ScalingConfig(
        base_lr=3e-4,
        base_batch_size=8,
        base_world_size=4,
        scaling_rule=ScalingRule.SQRT,
        warmup_steps=100,
    ),
    accum_config=GradAccumulationConfig(
        target_global_batch_size=512,
        local_batch_size=8,
        base_world_size=4,
    ),
    optimizer=dummy_optimizer,
)

print(f"Initial state (4 GPUs):")
print(f"  LR: {elastic_manager.current_lr:.2e}")
print(f"  Accumulation steps: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 4 * elastic_manager.accumulation_steps}")
print()

# Simulate losing 2 GPUs
print("Worker failure: 4 GPUs -> 2 GPUs")
elastic_manager.on_topology_change(2)
print(f"  New LR: {elastic_manager.current_lr:.2e}")
print(f"  New accumulation: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 2 * elastic_manager.accumulation_steps}")
print()

# Simulate scaling up to 8 GPUs
print("Scaling up: 2 GPUs -> 8 GPUs")
elastic_manager.on_topology_change(8)
print(f"  New LR: {elastic_manager.current_lr:.2e}")
print(f"  New accumulation: {elastic_manager.accumulation_steps}")
print(f"  Global batch: {8 * 8 * elastic_manager.accumulation_steps}")

## 10. Generate Sample Text

Use the trained model to generate text.

In [ ]:
@torch.no_grad()
def generate(model, tokenizer, prompt, max_new_tokens=50, temperature=0.8, device='cuda'):
    """Simple greedy/sampling text generation."""
    model.eval()
    
    input_ids = tokenizer.encode(prompt, return_tensors='pt').to(device)
    
    for _ in range(max_new_tokens):
        # Truncate if too long
        if input_ids.size(1) >= model.max_seq_length:
            input_ids = input_ids[:, -model.max_seq_length + 1:]
        
        outputs = model(input_ids)
        logits = outputs['logits'][:, -1, :] / temperature
        
        probs = torch.softmax(logits, dim=-1)
        next_token = torch.multinomial(probs, num_samples=1)
        
        input_ids = torch.cat([input_ids, next_token], dim=1)
        
        if next_token.item() == tokenizer.eos_token_id:
            break
    
    return tokenizer.decode(input_ids[0], skip_special_tokens=True)


# Generate some text
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

prompts = [
    "The history of",
    "In the year",
    "Scientists discovered that",
]

print("Text Generation Examples")
print("=" * 60)
for prompt in prompts:
    generated = generate(model, tokenizer, prompt, max_new_tokens=30, temperature=0.9, device=device)
    print(f"\nPrompt: {prompt}")
    print(f"Generated: {generated}")

## Summary

This notebook demonstrated large-scale training with `elastic_harness`:

| Feature | Details |
|---------|--------|
| Model | ~100M parameters (512d, 6 layers, 8 heads) |
| Data | WikiText-2 with GPT-2 tokenization |
| Training | 500 steps, batch 8 (via accumulation) |
| Checkpointing | Memory + NVMe multi-tier |
| Scaling | SQRT rule, 50-step warmup |
| Validation | Every 100 steps |

**Key elastic_harness components used:**
- `MemorySnapshotBackend` - Fast in-memory checkpoints
- `NVMeBackend` - Persistent disk checkpoints with cleanup
- `LRScalingManager` - Learning rate scaling on topology changes
- `ElasticScalingManager` - Unified LR + accumulation management

**For production deployment:**
- Add `S3Backend` for durable cloud checkpoints
- Use `CheckpointManager` for coordinated multi-tier saves
- Deploy etcd for multi-node elastic rendezvous
- Enable chaos testing to validate fault tolerance